In [2]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, ".")

import asyncio
from src.main import run_round
from src.file_util import load_data, save_data, safe_load_yaml

In [8]:
# enter round 1 through 6
asyncio.create_task(run_round(6))

Done with region: None, round: 6


In [9]:
df2 = load_data(region="west", rnd=1)

df2.head()

,team_name,seed,region,round
effective_seed,,,,
1,gonzaga,1,west,1
2,duke,2,west,1
3,texas_tech,3,west,1
4,arkansas,4,west,1
5,uconn,5,west,1
